# Import packages

In [90]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pathlib import Path

import xarray as xr
import pandas as pd
import numpy as np
import yaml
import matplotlib.pyplot as plt

from camels_spat2nh import *

# Get the current working directory of the notebook
CURRENT_DIR = os.getcwd()
# Add the parent directory of the notebook to the Python path
# root_dir = os.path.abspath(os.path.join(current_dir, '..', '..'))
ROOT_DIR = os.path.abspath(CURRENT_DIR)
sys.path.append(ROOT_DIR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
# Print packages version
print(f'xarray version: {xr.__version__}')
print(f'pandas version: {pd.__version__}')
print(f'numpy version: {np.__version__}')
print(f'yaml version: {yaml.__version__}')

xarray version: 2023.1.0
pandas version: 2.0.3
numpy version: 1.25.2
yaml version: 6.0.1


# Constants

In [92]:
basin_id = '14400000'
country = 'USA'

basin_folder = f'{country}_{basin_id}'

# Load Data

## Util data

In [93]:
data_dir_dict, data_gen_dict = load_util_data(CURRENT_DIR)
# display(
#     data_dir_dict,
#     data_gen_dict
# )

In [94]:
cspat_data_path = Path(data_dir_dict['data_dir_camels_spat']) / 'basin_data'
relative_path_forcing = data_dir_dict['relative_path_forcing']
countries = data_gen_dict['countries']
data_sources = data_gen_dict['data_sources']

# display(
#     cspat_data_path,
#     countries
# )

## Find basin and list of forcing files

In [95]:
# List all the folders in the basin_data directory
basin_data_folders = [str(f).split('/')[-1] for f in cspat_data_path.iterdir() if f.is_dir()]
# Check if basin_folder exists in the basin_data directory
if basin_folder not in basin_data_folders:
    raise FileNotFoundError(f'{basin_folder} folder not found in {cspat_data_path}')
else:
    print(f'{basin_folder} folder found in {cspat_data_path}')
    basin_folder_path = cspat_data_path / basin_folder / relative_path_forcing

USA_14400000 folder found in /project/gwf/gwf_cmt/wknoben/camels_spat/camels-spat-data/basin_data


In [96]:
# Group the files in the basin_folder by the data_source
# if contains data_source in the name
basin_files = {}
for data_source in data_sources:
    basin_files[data_source] = sorted([f for f in basin_folder_path.iterdir() if data_source in str(f)])

# Print number of files in each data_source
for data_source in data_sources:
    print(f'Number of files in {data_source}: {len(basin_files[data_source])}')

Number of files in ERA5: 640
Number of files in EM_Earth: 603
Number of files in daymet: 44
Number of files in RDRS: 468


In [97]:
# Select the first file for each data_source
basin_data = {}
for data_source in data_sources:
    basin_data[data_source] = xr.open_dataset(basin_files[data_source][0])

In [98]:
# Display the data variables in the basin_data
for data_source in data_sources:
    display(data_source, basin_data[data_source])
    # List of Data Variables
    print(f'Data Variables in {data_source}: {list(basin_data[data_source].data_vars)}')

'ERA5'

<xarray.Dataset>
Dimensions:    (time: 744, hru: 1, nbnds: 2)
Coordinates:
  * time       (time) datetime64[ns] 1969-09-30T16:00:00 ... 1969-10-31T15:00:00
  * nbnds      (nbnds) int32 1 2
Dimensions without coordinates: hru
Data variables: (12/19)
    e          (time, hru) float32 ...
    hruId      (hru) float64 ...
    latitude   (hru) float64 ...
    longitude  (hru) float64 ...
    mper       (time, hru) float32 ...
    msdwlwrf   (time, hru) float32 ...
    ...         ...
    t          (time, hru) float32 ...
    u          (time, hru) float32 ...
    v          (time, hru) float32 ...
    w          (time, hru) float32 ...
    phi        (time, hru) float32 ...
    time_bnds  (nbnds, time) datetime64[ns] ...
Attributes:
    Conventions:  CF-1.6
    Author:       The data were written by CAMELS-spat workflow
    License:      
    Source:       Case: ERA5_lumped; remapped by script from library of Sherv...
    NCO:          netCDF Operators version 5.0.6 (Homepage = http://nco.sf.ne...
    History:      Thu Jan  4 08:24:12 2024: ncks -x -v reflected_sw,net_radia...

'EM_Earth'

<xarray.Dataset>
Dimensions:    (time: 744, hru: 1, nbnds: 2)
Coordinates:
  * time       (time) datetime64[ns] 1969-09-30T16:00:00 ... 1969-10-31T15:00:00
  * nbnds      (nbnds) int32 1 2
Dimensions without coordinates: hru
Data variables:
    latitude   (hru) float64 ...
    longitude  (hru) float64 ...
    hruId      (hru) float64 ...
    tmean      (time, hru) float32 ...
    prcp       (time, hru) float32 ...
    time_bnds  (nbnds, time) datetime64[ns] ...
Attributes:
    Conventions:  CF-1.6
    Author:       The data were written by CAMELS-spat workflow
    License:      
    Source:       Case: EM-Earth_lumped; remapped by script from library of S...
    History:      Created Fri Sep 22 01:13:57 2023  On Fri Jan 12 11:20:16 20...

'daymet'

<xarray.Dataset>
Dimensions:    (time: 366, hru: 1, nv: 2)
Coordinates:
  * time       (time) datetime64[ns] 1980-01-01T12:00:00 ... 1980-12-31T12:00:00
Dimensions without coordinates: hru, nv
Data variables:
    latitude   (hru) float64 ...
    longitude  (hru) float64 ...
    hruId      (hru) float64 ...
    prcp       (time, hru) float32 ...
    tmax       (time, hru) float32 ...
    tmin       (time, hru) float32 ...
    srad       (time, hru) float32 ...
    vp         (time, hru) float32 ...
    dayl       (time, hru) float32 ...
    pet        (time, hru) float32 ...
    time_bnds  (time, nv) datetime64[ns] ...
Attributes:
    Conventions:    CF-1.6
    Author:         The data were written by CAMELS-spat workflow
    License:        GPLv3+; Originla license: No license from original file i...
    History:        Created Wed Jun 19 22:49:50 2024by EASYMORE nc_remapper; ...
    easymore_hash:  835a10df80cccc9ceaef12a22497986cfa7f87a6f15cba6e6a34f7276...
    Source:         Remapped by EASYMORE nc_remapper from original file: /gpf...

'RDRS'

<xarray.Dataset>
Dimensions:              (time: 744, hru: 1, nbnds: 2)
Coordinates:
  * time                 (time) datetime64[ns] 1980-01-01T05:00:00 ... 1980-0...
  * nbnds                (nbnds) int32 1 2
Dimensions without coordinates: hru
Data variables: (12/18)
    latitude             (hru) float64 ...
    longitude            (hru) float64 ...
    hruId                (hru) float64 ...
    RDRS_v2.1_A_PR0_SFC  (time, hru) float32 ...
    RDRS_v2.1_P_FB_SFC   (time, hru) float32 ...
    RDRS_v2.1_P_FI_SFC   (time, hru) float32 ...
    ...                   ...
    RDRS_v2.1_P_UVC_10m  (time, hru) float32 ...
    RDRS_v2.1_P_VVC_10m  (time, hru) float32 ...
    e                    (time, hru) float32 ...
    pet                  (time, hru) float32 ...
    time_bnds            (nbnds, time) datetime64[ns] ...
    phi                  (time, hru) float32 ...
Attributes:
    Conventions:    CF-1.6
    Author:         The data were written by CAMELS-spat workflow
    License:        GPLv3+; Originla license:  License: These data are provid...
    easymore_hash:  c27e9ebe7aa15fb0a3731c77f6e07f5a2e2e007dfa904698fc941b8a1...
    Source:         Remapped by EASYMORE nc_remapper from original file: /gpf...
    History:        Created Fri Jul 26 16:38:57 2024by EASYMORE nc_remapper; ...